# This is the code for lab 5

### **Produce a table for each riboswitch including riboswitch name, start and end coordinates of OBS-1, start and end coordinates of OBS-2 and start and end coordinates of the two red regions**
The sequence of DNA is input manually:
_**AND-1**_
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU
.............................xxxxxxxxxxxxxxxx...xxxxxxxxxxxxxxxx................................................
.............................xxxxxxxxxxxxxxxx...................................................................
................................................xxxxxxxxxxxxxxxx................................................

_**OR-1**_
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
..........................xxxxxxxxxxxxxxxxxxxx.........................................................
..............................................xxxxxxxxxxxxxxxxxxxx.....................................
..........................xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.....................................

_**NOT-1**_
GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUGCGUCCUGUAUUCCACUGC
...........................................xxxxxxxxxxxxxxxxxxxxxxx..........................

_**YES-1**_
GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
.........................xxxxxxxxxxxxxxxxxxxxxx.................................




### Answer the pre-questions before hunting the RNAfold code
_Will the AND-1 riboswitch cleave itself when both of its OBS are bound?_ 
Yes, it will. For AND logic function,it is only active when receiving two separate molecular impulse, in this case, two separate DNA oligonucleotides.

_Will the OR-1 riboswitch cleave itself when neither of its OBS are bound?_
As a rule, it won't. And from the result of the experiment, it actually showed little self-cleavage in the absence of effector DNAs as predicted.

_What behavior do we expect from the YES-1 riboswitch?_
YES-1 is predicted to form the desired OFF- and ON-state strctures in the absence and presence, respectively, of a 22-nt effector DNA.

In [1]:
import sqlite3
conn=sqlite3.connect('riboswitch.db')
c = conn.cursor()

c.execute("""CREATE TABLE switch
             (name TEXT, start_OBS1 INT, end_OBS1 INT,start_OBS2 INT, end_OBS2 INT,start_RED1 INT, end_RED1 INT,start_RED2 INT, end_RED2 INT);""")
c.execute("""INSERT INTO switch VALUES ('AND-1',30,45,49,64,16,23,70,77);""")
c.execute("""INSERT INTO switch VALUES ('OR-1',27,46,47,66,16,26,67,77);""")
c.execute("""INSERT INTO switch (name,start_OBS1,end_OBS1,start_RED1,end_RED1,start_RED2,end_RED2) VALUES ('NOT-1',44,66,40,43,74,77);""")
c.execute("""INSERT INTO switch (name,start_OBS1,end_OBS1,start_RED1,end_RED1,start_RED2,end_RED2) VALUES ('YES-1',26,47,16,21,49,54);""")
conn.commit()
t = ('NOT-1',)
c.execute('SELECT * FROM switch WHERE name=?', t)
print(c.fetchone())


('NOT-1', 44, 66, None, None, 40, 43, 74, 77)


### Write a routine to fold RNA, then generate one plot per riboswitch

In [17]:
import subprocess
seqs=\
""">AND-1
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU\n
>OR-1
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC\n
>NOT-1
GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUGCGUCCUGUAUUCCACUGC\n
>YES-1
GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
@
"""
p = subprocess.run(['RNAfold'],input=bytes(seqs,'ascii'),stdout = subprocess.PIPE,stderr = subprocess.PIPE)
print(p.stderr.decode())

### Comparing each plot to the native conformation given in the publication
1.AND-1
They are the same!
2.OR-1
They are the same!
3.NOT-1
They are the same!
4.YES-1
They are different! Region IV, which is stem structure in the publication, become loop structure in the plot. The bottom loop structure in the publication becomes stem structure in the plot. In both structures, the red regions are not bound each other. However, in the published structure, the red regions are in the same stem, while they are in the different stem/loop at the plot I drew.
They parameters used in the two method may vary.

### Simulating OBS binding in YES-1 and NOT-1
The following code would generate 2 plots corresponding to YES-1 and NOT-1 structure. 

In [16]:
import sys
infile=open("NOT-1-Constrained.txt")
line=infile.read()
p = subprocess.run(['RNAfold','-C'],input=bytes(line,"ascii"),stdout = subprocess.PIPE,stderr = subprocess.PIPE)
print(p.stderr.decode())

In [18]:
import sys
infile=open("YES-1-Constrained.txt")
line=infile.read()
p = subprocess.run(['RNAfold','-C'],input=bytes(line,"ascii"),stdout = subprocess.PIPE,stderr = subprocess.PIPE)
print(p.stderr.decode())

1. NOT-1-OBS
They are the same! The red regions do not bound each otehr.
2. YES-1-OBS
They are the same! And the red regions bound each other.

### Simulating OBS binding in AND-1 and OR-1

In [24]:
names=['AND-1-OBS1-Constrained','AND-1-OBS2-Constrained','AND-1-OBS1_2-Constrained','OR-1-OBS1-Constrained','OR-1-OBS2-Constrained','OR-1-OBS1_2-Constrained']
for name in names:
    infile=open(name+'.txt')
    line=infile.read()
    p = subprocess.run(['RNAfold','-C'],input=bytes(line,"ascii"),stdout = subprocess.PIPE,stderr = subprocess.PIPE)
    print(p.stderr.decode())

### Determine whether the red regions pair with each other for each of the four possible input combination 
Notation: F/T represents OBS-1 is not bound to complementary DNA while OBS-2 is bound to complementary DNA. Additionally, if the red regions are paird, then output "True", otherwise, output "False"
1. AND-1
F/F -> False
T/F -> False, but the stem-loop structure from my plot is different from the published structure.
F/T -> False, but the stem-loop structrue from my plot is different from the published structure.
T/T -> False, the stem-loop structure from my plot is different from the published structure.

2. OR-1
F/F -> False, the stem-loop structure plotted here is same as the published one.
T/F -> False, thers is an extra loop structure near the in the red regions from my plot.
F/T -> True, they are the same!
T/T -> True, they are the same!

According to my result, the design of AND-1 disagree the logic of AND gate and doesn't activate self-clevage when two separate molecular impulses input together.
The design of OR-1 only partially satisfies the OR gate. Self-clevage will only be activated when only OBS-2 region is bound to oligonucleotide2 impulse, no matter in the presence/absence of oligonucleotide 1.